In [17]:
import time,serial
import numpy as np

def servoin(a=90,b=90,c=90,d=90,e=90,f=90,delay=3):
    '''waits end of previous act. and send angles to arduino returns acc-array     
    '''
    while ser.read()!=b'>':pass
    acc = [float(e) for e in ser.readline().split(b'\t')]
    print(acc)
    with open('historia.log', 'a') as myfile:
            myfile.write('Accelerometer:'+str(acc)+'\n')
    if fall_check(*acc)==False:
        data='in'
        for i in a,b,c,d,e,f,delay:
            try: 
                data+=str(hex(i))[2]+str(hex(i))[3]
            except IndexError:
                data+='0'+str(hex(i))[2]
        ser.write(data.encode())
        print('servo in: ',a,b,c,d,e,f,delay)
        with open('historia.log', 'a') as myfile:
            myfile.write('Servo in:'+str([a,b,c,d,e,f,delay])+'\n')
    else:
         1/0
    return acc
    
def fall_check(accelx,accely,accelz,gx,gy,gz):
    '''gets measurement of accelerometer
        checks if Robo has fallen
        returns FALSE if Robo - NOT fallen
        returns TRUE if Robo - has fallen'''
    if (abs(accely)>5 or abs(accelz)>5) and abs(accelx)<7:return True
    else: return False
    
     
def serial_begin(port):
    ser = serial.Serial(port, 19200, bytesize=8, parity='N', stopbits=1, timeout=2)
    time.sleep(2)
    print('connected to: '+ ser.portstr)
    with open('historia.log', 'a') as myfile:
            myfile.write(time.ctime()+'\nconnected to:'+str(ser.portstr)+'\n')
    return ser,np.array([[90,90,90,90,90,90]])

def executor(ser,prt,mtrx):
    k = 0
    acc = []
    try:#fall detection script            
        for i in mtrx: 
            acc.append(servoin(*i))
            k += 1
    except ZeroDivisionError:
        print('I have fallen.')
        with open('historia.log', 'a') as myfile:
            myfile.write('I have fallen\n')
        stand_up([90,90,90,90,90,90])
    return ser,k,acc

def stand_up(old):
    """TRUE when ready"""
    try:
        data='in'
        for i in old:
            try: 
                data+=str(hex(i))[2]+str(hex(i))[3]
            except IndexError:
                data+='0'+str(hex(i))[2]
        data+='03'
        ser.write(data.encode())
        servoin(*old)
    except ZeroDivisionError:
        print('Master, lift me up, please...')
        stand_up(old)

if __name__ == "__main__":
    port = 'COM9'
    ser,matrix = serial_begin(port)
    matrix = np.loadtxt('matrix.txt', 'int')
    ser,stf,acel_data = executor(ser,port,matrix)
    if stf < len(matrix): print('Steps to fall :', stf)
    with open('historia.log', 'a') as myfile:
            myfile.write('Steps to fall:'+str(stf)+'\n')
    np.savetxt('acc.txt',acel_data,fmt='%.2f',delimiter='\t')
    with open('historia.log', 'a') as myfile:
            myfile.write('[Upd]acc.txt\nAccelerometer data recieved.\n')    
    ser.close()            

connected to: COM9
[-10.04, -1.14, -0.86, 0.0, 0.0, 0.0]
servo in:  90 90 90 90 90 90 3
[-9.73, -0.04, -0.59, 0.24, 0.43, 0.69]
servo in:  90 90 90 90 80 90 3
[-10.63, -1.26, -0.86, 1.29, 0.32, 1.73]
servo in:  70 70 90 90 80 80 3
[-10.04, 2.24, -5.77, 20.94, -236.24, -0.89]
servo in:  50 70 90 90 80 80 3
[-9.69, 0.67, 3.88, -21.24, -6.17, 4.88]
servo in:  50 70 70 70 90 90 3
[-9.96, -3.33, 2.0, -31.66, 104.24, 74.55]
servo in:  90 100 70 90 110 110 3
[-9.69, -3.1, 1.69, 62.91, 14.66, -23.65]
servo in:  90 120 100 90 120 110 3
[-9.45, -3.45, 2.79, -89.29, 3.01, -8.42]
servo in:  120 130 120 110 120 110 3
[-8.16, -1.92, -2.94, 60.51, -9.67, -1.11]
servo in:  120 130 140 110 90 90 3
[-6.51, -1.88, -2.24, -95.49, -107.95, 46.11]
servo in:  60 90 80 80 80 80 3


In [12]:
ser.close()